for the moment, uncomment out the parts you need to run. if first time running, run all

### 0 : importing packages

In [1]:
import json
from pathlib import Path, PurePath 
import glob

import betfairlightweight
from betfairlightweight import filters

import datetime

import pandas as pd
import numpy as np

from bz2 import BZ2File 

from betfairlightweight import StreamListener
from betfairlightweight.streaming.stream import MarketStream

import data_utils
from data_utils import *

### 1 : 'logging in'

In [2]:
project_dir = Path.cwd().parents[1]
logins_dir = project_dir / 'api_logins.json'

with open(logins_dir) as f:
    login_dict =  json.load(f)
    
trading = betfairlightweight.APIClient(username=login_dict['my_username'],
                                       password=login_dict['my_password'],
                                       app_key=login_dict['my_app_key'],
                                       certs=login_dict['certs_path'])

trading.login()

<LoginResource>

### 2 : retrieving api data list

In [6]:
# # returns list of 'data dictionaries'
# data_dicts = trading.historic.get_my_data()

# # calculate range of dates for advanced data
# adv_range = [d['forDate'] for d in data_dicts if d['plan'] == 'Advanced Plan']

# # find min date for adv_data
# adv_min_date = datetime.datetime.strptime(min(adv_range), '%Y-%m-%dT%H:%M:%S')

# # find max data for adv data
# def last_day_of_month(any_day):
#     next_month = any_day.replace(day=28) + datetime.timedelta(days=4)  
#     return next_month - datetime.timedelta(days=next_month.day)

# adv_max_temp = datetime.datetime.strptime(max(adv_range), '%Y-%m-%dT%H:%M:%S')
# adv_max_date = last_day_of_month(adv_max_temp) 

In [7]:
# # list files within advanced data range (GB Data)
# adv_file_list = trading.historic.get_file_list(
#     "Horse Racing",
#     "Advanced Plan",
#     from_day=adv_min_date.day,
#     from_month=adv_min_date.day,
#     from_year=adv_min_date.year,
#     to_day=adv_max_date.day,
#     to_month=adv_max_date.month,
#     to_year=adv_max_date.year,
#     market_types_collection=["WIN"],
#     countries_collection=["GB"],
#     file_type_collection=["M"],
# )
# print("No. items :", len(adv_file_list))

No. items : 1858


### 3 : downloading data
DON'T interrupt this when downloading or file may come corrupted and cause issues when processing

In [8]:
# # # where to store our advanced data
# raw_dir = project_dir / 'data' / 'raw' / 'api' / 'advanced'

# # # check if files have been downloaded already
# adv_all_files = [Path(f).name for f in adv_file_list] # all files to download
# adv_downloaded_files = [Path(f).name for f in raw_dir.glob("*.bz2")] # all files downloaded

# # download files we dont have and writing uncompressed versions
# for file in adv_file_list: # remove index for all files
#     if Path(file).name not in adv_downloaded_files: 
#         print(file)
#         download = trading.historic.download_file(file_path = file, store_directory = raw_dir)
#         print(download)

/xds_nfs/edp_processed/ADVANCED/2020/Jan/1/29637009/1.166897853.bz2
C:\Users\Asus\OneDrive\python\betfair_project\data\raw\api\advanced\1.166897853.bz2
/xds_nfs/edp_processed/ADVANCED/2020/Jan/1/29637009/1.166897858.bz2
C:\Users\Asus\OneDrive\python\betfair_project\data\raw\api\advanced\1.166897858.bz2
/xds_nfs/edp_processed/ADVANCED/2020/Jan/1/29637065/1.166898709.bz2
C:\Users\Asus\OneDrive\python\betfair_project\data\raw\api\advanced\1.166898709.bz2
/xds_nfs/edp_processed/ADVANCED/2020/Jan/1/29637065/1.166898714.bz2
C:\Users\Asus\OneDrive\python\betfair_project\data\raw\api\advanced\1.166898714.bz2
/xds_nfs/edp_processed/ADVANCED/2020/Jan/1/29637065/1.166898719.bz2
C:\Users\Asus\OneDrive\python\betfair_project\data\raw\api\advanced\1.166898719.bz2
/xds_nfs/edp_processed/ADVANCED/2020/Jan/1/29637065/1.166898724.bz2
C:\Users\Asus\OneDrive\python\betfair_project\data\raw\api\advanced\1.166898724.bz2
/xds_nfs/edp_processed/ADVANCED/2020/Jan/1/29637065/1.166898729.bz2
C:\Users\Asus\OneDri

### 4 : processing data

decompressing and writing

In [11]:
# #proceesing bz2 to text (try/except for thos files corrupted when downloaded, not many)
# adv_extfile_dirs = []

# for file in glob.glob(str(raw_dir) + '/*.bz2'): # change this?
#     try:
#         zipfile = BZ2File(file) # open the file
#         data = zipfile.read() # get the decompressed data
#         newfilepath = file.split('.bz2')[0] # removing the extension and saving without a filetype
#         open(newfilepath, 'wb').write(data) # write an uncompressed file
#         adv_extfile_dirs.append(newfilepath)
#         zipfile.close()
#     except OSError:
#         print("File not processed : ", file)
#         pass
    
# print("EX DONE")

File not processed :  C:\Users\Asus\OneDrive\python\betfair_project\data\raw\api\advanced\1.166898714.bz2
File not processed :  C:\Users\Asus\OneDrive\python\betfair_project\data\raw\api\advanced\1.167074916.bz2
File not processed :  C:\Users\Asus\OneDrive\python\betfair_project\data\raw\api\advanced\1.167074936.bz2
File not processed :  C:\Users\Asus\OneDrive\python\betfair_project\data\raw\api\advanced\1.168902625.bz2
EX DONE


In [12]:
# add code in to then redownload these files that are corrupted??

### 5 : 'streaming' / writing processed data

In [13]:
processed_files = glob.glob(str(raw_dir) + '/*[!.bz2]')

In [14]:
datadict = {'Time': [],
       'MarketId' : [],
       'Status' : [],
       'Inplay' : [], 
       'SelectionId' : [],
       'LastPriceTraded' : [],
       'TotalMatched' : [],
       'BSP' : [],
       'AdjFactor' :  [],
       'RunnerStatus' : [],
       'MktTotalMatched' : [],
       'RaceInfo' : [],
       'Venue' : [],
       'BackSize': [],
       'BackPrice': [],
       'LayPrice' : [],
       'LaySize' : []
}

d = datadict

In [15]:
class HistoricalStream(MarketStream):
    # create custom listener and stream

    def _init_(self, listener):
        super(HistoricalStream, self)._init_(listener)

    def on_process(self, market_books):
        for market_book in market_books:
            for runner in market_book.runners:

                datadict['Time'].append(market_book.publish_time)
                datadict['MarketId'].append(float(market_book.market_id))
                datadict['Status'].append(market_book.status)
                datadict['Inplay'].append(market_book.inplay)
                datadict['SelectionId'].append(runner.selection_id)
                datadict['LastPriceTraded'].append(runner.last_price_traded)
                datadict['TotalMatched'].append(runner.total_matched)
                datadict['BSP'].append(runner.sp.actual_sp)
                datadict['AdjFactor'].append(runner.adjustment_factor)
                datadict['RunnerStatus'].append(runner.status)
                datadict['MktTotalMatched'].append(market_book.total_matched)
                datadict['RaceInfo'].append(market_book.market_definition.name)
                datadict['Venue'].append(market_book.market_definition.venue)
                
                atb_size = [x.size for x in runner.ex.available_to_back]
                datadict['BackSize'].append(atb_size)
                atb_price = [x.price for x in runner.ex.available_to_back]
                datadict['BackPrice'].append(atb_price)   
                atl_price = [x.price for x in runner.ex.available_to_lay]
                datadict['LayPrice'].append(atl_price)
                atl_size = [x.size for x in runner.ex.available_to_lay]
                datadict['LaySize'].append(atl_size)


class HistoricalListener(StreamListener):
    def _add_stream(self, unique_id, stream_type):
        if stream_type == "marketSubscription":
            return HistoricalStream(self)

In [16]:
listener = HistoricalListener(max_latency=None)

In [17]:
final_df = pd.DataFrame()
interim_dir = project_dir / 'data' / 'interim' / 'api' / 'advanced'

T_PRE = 60
T_POST = 15


for file in processed_files:
    try:
        datadict = {'Time': [],
           'MarketId' : [],
           'Status' : [],
           'Inplay' : [], 
           'SelectionId' : [],
           'LastPriceTraded' : [],
           'TotalMatched' : [],
           'BSP' : [],
           'AdjFactor' :  [],
           'RunnerStatus' : [],
           'MktTotalMatched' : [],
           'RaceInfo' : [],
           'Venue' : [],
           'BackSize': [],
           'BackPrice': [],
           'LayPrice' : [],
           'LaySize' : []
        }


        stream = trading.streaming.create_historical_stream(directory=file, listener=listener)
        stream.start() 
        stream.stop()

        print(str(file) + " stream completed.")

        # create df
        df = dict_to_df(datadict)

        df = pre_filter_races(df);
        df = extract_race_info(df);
        df = create_timedif(df);
        df = filter_timedif(df);
        df = create_time_bins(df, T_PRE, T_POST);
        df = runner_groupby(df, T_PRE, T_POST)

        # write / read from here

        final_df = pd.concat([final_df, df])
        print('Rows added : ', len(df.index), 'Total rows : ', len(final_df.index))
        del df
        

    except:
        print("File not processed : ", file)
        pass
    
print("STREAM DONE")

C:\Users\Asus\OneDrive\python\betfair_project\data\raw\api\advanced\1.166897828 stream completed.
Rows added :  7 Total rows :  7
C:\Users\Asus\OneDrive\python\betfair_project\data\raw\api\advanced\1.166897833 stream completed.
Rows added :  3 Total rows :  10
C:\Users\Asus\OneDrive\python\betfair_project\data\raw\api\advanced\1.166897838 stream completed.
Rows added :  7 Total rows :  17
C:\Users\Asus\OneDrive\python\betfair_project\data\raw\api\advanced\1.166897843 stream completed.
Rows added :  5 Total rows :  22
C:\Users\Asus\OneDrive\python\betfair_project\data\raw\api\advanced\1.166897848 stream completed.
Rows added :  9 Total rows :  31
C:\Users\Asus\OneDrive\python\betfair_project\data\raw\api\advanced\1.166897853 stream completed.
Rows added :  6 Total rows :  37
C:\Users\Asus\OneDrive\python\betfair_project\data\raw\api\advanced\1.166897858 stream completed.
Rows added :  7 Total rows :  44
C:\Users\Asus\OneDrive\python\betfair_project\data\raw\api\advanced\1.166898709 stre

C:\Users\Asus\AppData\Roaming\Python\Python37\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


Rows added :  0 Total rows :  9807
C:\Users\Asus\OneDrive\python\betfair_project\data\raw\api\advanced\1.169392603 stream completed.
Rows added :  0 Total rows :  9807
C:\Users\Asus\OneDrive\python\betfair_project\data\raw\api\advanced\1.169393609 stream completed.
Rows added :  14 Total rows :  9821
C:\Users\Asus\OneDrive\python\betfair_project\data\raw\api\advanced\1.169393614 stream completed.
Rows added :  3 Total rows :  9824
C:\Users\Asus\OneDrive\python\betfair_project\data\raw\api\advanced\1.169393619 stream completed.
Rows added :  7 Total rows :  9831
C:\Users\Asus\OneDrive\python\betfair_project\data\raw\api\advanced\1.169393624 stream completed.
Rows added :  10 Total rows :  9841
C:\Users\Asus\OneDrive\python\betfair_project\data\raw\api\advanced\1.169393629 stream completed.
Rows added :  11 Total rows :  9852
C:\Users\Asus\OneDrive\python\betfair_project\data\raw\api\advanced\1.169393634 stream completed.
Rows added :  9 Total rows :  9861
C:\Users\Asus\OneDrive\python\b

In [20]:
final_df.head()

,SelectionId,MarketId,Venue,Distance,RaceType,BSP,NoRunners,BS:T-60,BS:T-59,BS:T-58,...,LS:T+5,LS:T+6,LS:T+7,LS:T+8,LS:T+9,LS:T+10,LS:T+11,LS:T+12,LS:T+13,LS:T+14
0,11688029,1.166897828,Southwell,8.0,Flat,9.20,7,4.15,5.98,6.86,...,4.76,7.70,3.07,41.07,8.05,3.74,1.85,7.05,3.89,0.41
1,13331255,1.166897828,Southwell,8.0,Flat,4.30,7,41.50,64.89,38.54,...,16.44,7.38,18.12,5.44,4.09,15.50,3.82,66.43,192.93,136.06
2,13663992,1.166897828,Southwell,8.0,Flat,6.01,7,4.99,5.96,22.29,...,5.65,8.66,9.03,9.89,15.51,19.78,161.00,3.34,1.84,1.49
3,19252822,1.166897828,Southwell,8.0,Flat,3.94,7,19.08,21.03,15.03,...,4.81,40.84,17.37,24.52,11.81,43.40,4.59,7.87,1.51,1.24
4,19436245,1.166897828,Southwell,8.0,Flat,40.00,7,4.34,3.72,4.93,...,26.16,2.00,20.63,2.45,7.77,1.09,3.49,2.67,0.44,2.27


In [21]:
final_df.shape

(12903, 307)

### 6 : writing out data

In [22]:
processed_dir = project_dir / 'data' / 'processed' / 'api' / 'advanced'
final_df.to_csv(str(processed_dir) + '/adv_data.csv')

In [ ]:
# To-Do
# use multiindexing for columns for time points / prices/ sizes in future?
# move .py version to src/data/historic
# investigate nulls befre removal in filtering
# define 'hard variables' with capitals e.g. directories

# add doc strings to each function e.g. qcut
# include irish/us racing also

# add in other variables like no.runners, country, date

# add in conditions to make script 'run beackawards' e.g.
# - if market name not in final_df file then stream - and add it in - call this updates?

# then wat to do with unprocessed files, redownload these?